In [137]:
%load_ext autoreload
%autoreload 2

import os
import sys


from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU
import numpy as np
import pandas as pd


sys.path.append('../src')

import enigma_challenge as ec

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate dataset

In [221]:
# create dataset of len 2**15
power = 15
n_samples = 1<<power

# if the data is already saved, reads the data, otherwise generates the data
save_file = f"../data/raw/enigma_data_{power}.csv"
ENIGMA_OBJ = ec.EnigmaDataset(n_samples=n_samples, seq_len=42, save_file=save_file)
ENIGMA_OBJ.dataset.head()

,PLAIN,CIPHER
0,GOLEARNABLEONE,HMSKLWYPLPTEVO
1,CARRYOTHERSASSUMEMEET,VEXBNYVOCGFGFXVBUCCIO
2,SELLNONEWITHACCEPT,UASHYYYVMHEMDWOGKH
3,GREENAIRBLACKLINEPUSHPAY,HGVKYTXDLPKUWVFYUVLRAHJW
4,PASSSEASONRESPONDMAYABLELESS,OELWWZOWSSUOFMCYHCDNHADNKXYY


# Test/train split (with caching)

In [222]:
n_test_samples = 16384
sent_partition_size = 7

ENIGMA_OBJ.test_train_split(n_test_samples=n_test_samples, 
                            sent_partition_size=sent_partition_size)

In [223]:
ENIGMA_OBJ.train_data_partitioned.head()

,ID,PLAIN,CIPHER
0,1,CARRYOT,VEXBNYV
1,1,HERSASS,OCGFGFX
2,1,UMEMEET,VBUCCIO
3,2,SELLNON,UASHYYY
4,2,EWITHAC,VMHEMDW


In [224]:
ENIGMA_OBJ.test_data_partitioned.head()

,ID,PLAIN,CIPHER
0,1588,FINALLY,JFPIAUN
1,1588,SUMMERA,WXYCOMH
2,1588,PPROACH,BZJKDVA
3,1588,TENHOTE,XOENYOH
4,1588,LSHAKE,KIDCWL


In [225]:
print(f"# of training examples: {ENIGMA_OBJ.train_data.shape[0]}")
print(f"# of test examples: {ENIGMA_OBJ.test_data.shape[0]}")

# of training examples: 16384
# of test examples: 16384


# Encode the dataset

In [226]:
ENIGMA_ENCODED = ec.EncodedDataset(unencoded_dataset=ENIGMA_OBJ)

In [227]:
print(ENIGMA_ENCODED.plain_train.sentences_processed[0:5])
print(ENIGMA_ENCODED.cipher_train.sentences_processed[0:5])
print(ENIGMA_ENCODED.plain_test.sentences_processed[0:5])
print(ENIGMA_ENCODED.cipher_test.sentences_processed[0:5])

['\tCARRYOT\n', '\tHERSASS\n', '\tUMEMEET\n', '\tSELLNON\n', '\tEWITHAC\n']
['VEXBNYV', 'OCGFGFX', 'VBUCCIO', 'UASHYYY', 'VMHEMDW']
['\tFINALLY\n', '\tSUMMERA\n', '\tPPROACH\n', '\tTENHOTE\n', '\tLSHAKE\n']
['JFPIAUN', 'WXYCOMH', 'BZJKDVA', 'XOENYOH', 'KIDCWL']


In [228]:
print(ENIGMA_ENCODED.plain_train.alphabet)
print(ENIGMA_ENCODED.cipher_train.alphabet)
print(ENIGMA_ENCODED.plain_test.alphabet)
print(ENIGMA_ENCODED.cipher_test.alphabet)

['\t', '\n', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
['\t', '\n', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [229]:
print(ENIGMA_ENCODED.plain_train.target_vector)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


# Create the Ultra Code Breaker model

In [230]:
ultra = ec.UltraCodeBreaker(encoded_dataset=ENIGMA_ENCODED)

# Train the model

In [232]:
ultra.train(epochs=50, n_nodes=256)
ultra.model.summary()

Train on 46674 samples, validate on 20004 samples
Epoch 1/50
46674/46674 [==============================] - 18s 385us/step - loss: 2.1711 - val_loss: 2.2266
Epoch 2/50
46674/46674 [==============================] - 16s 350us/step - loss: 1.9999 - val_loss: 1.8813
Epoch 3/50
46674/46674 [==============================] - 16s 349us/step - loss: 1.7393 - val_loss: 1.6489
Epoch 4/50
46674/46674 [==============================] - 18s 387us/step - loss: 1.6326 - val_loss: 1.5361
Epoch 5/50
46674/46674 [==============================] - 17s 361us/step - loss: 1.4634 - val_loss: 1.4055
Epoch 6/50
46674/46674 [==============================] - 16s 351us/step - loss: 1.3405 - val_loss: 1.2981
Epoch 7/50
46674/46674 [==============================] - 16s 337us/step - loss: 1.2413 - val_loss: 1.1885
Epoch 8/50
46674/46674 [==============================] - 16s 337us/step - loss: 1.1247 - val_loss: 1.0999
Epoch 9/50
46674/46674 [==============================] - 18s 378us/step - loss: 1.0244 - val_

# Create the test model

In [233]:
ultra.create_test_model(n_nodes=256)

# Predict

In [234]:
ENIGMA_OBJ.test_data_partitioned.head()

,ID,PLAIN,CIPHER
0,1588,FINALLY,JFPIAUN
1,1588,SUMMERA,WXYCOMH
2,1588,PPROACH,BZJKDVA
3,1588,TENHOTE,XOENYOH
4,1588,LSHAKE,KIDCWL


# Evaluate the test predictions

In [239]:
# save_file = "../data/RUN_DATA/enigma_predicted_n7_79.csv"
# save_file = './enigma_predicted_test.csv'
ultra.predict()

In [ ]:
ultra.evaluate('../data/processed/predictions_n7_acc77')